In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import math
import missingno as msno

# Marlon's stuk

In [2]:
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print ("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")

In [3]:
tic()
df = pd.read_csv("small_data/50000_training_set_VU_DM.csv")
toc()

# Code to replace missing prop_review_score values with corresponding propstarrating vlaues.
def fill_prop_review_score(df):
    for i in range(len(df)):
        if math.isnan(df['prop_review_score'][i]):
            df['prop_review_score'][i] = df['prop_starrating'][i]
    return(df)
tic()
df = fill_prop_review_score(df)
toc()

Elapsed time is 0.807837963104248 seconds.


<ipython-input-3-d966af75fcf3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prop_review_score'][i] = df['prop_starrating'][i]


Elapsed time is 0.371518611907959 seconds.


In [4]:
tic()
# Code which adds columns with the average rate,inventory, and percentage difference between Expedia and competitors.
def make_rates(data):
    data['comp_rate'] = data[['comp1_rate',
                            'comp2_rate',
                            'comp3_rate',
                            'comp4_rate',
                            'comp5_rate',
                            'comp6_rate',
                            'comp7_rate',
                            'comp8_rate']].mean(axis=1)
    data['comp_rate'] = data['comp_rate'].fillna(0)
    
    data['comp_inv'] = data[['comp1_inv',
                            'comp2_inv',
                            'comp3_inv',
                            'comp4_inv',
                            'comp5_inv',
                            'comp6_inv',
                            'comp7_inv',
                            'comp8_inv']].mean(axis=1)
    data['comp_inv'] = data['comp_inv'].fillna(0)
    
    data['comp_percent'] = data[['comp1_rate_percent_diff',
                            'comp2_rate_percent_diff',
                            'comp3_rate_percent_diff',
                            'comp4_rate_percent_diff',
                            'comp5_rate_percent_diff',
                            'comp6_rate_percent_diff',
                            'comp7_rate_percent_diff',
                            'comp8_rate_percent_diff']].mean(axis=1)
    data['comp_percent'] = data['comp_percent'].fillna(0)
    
    return(data)
df = make_rates(df)
toc()

Elapsed time is 0.07480001449584961 seconds.


In [5]:
tic()
# Dorp column 'prop_location_score2', too many missing values.
df = df.drop(columns=['comp1_rate',
                 'comp2_rate',
                 'comp3_rate',
                 'comp4_rate',
                 'comp5_rate',
                 'comp6_rate',
                 'comp7_rate',
                 'comp8_rate',
                 'comp1_inv',
                 'comp2_inv',
                 'comp3_inv',
                 'comp4_inv',
                 'comp5_inv',
                 'comp6_inv',
                 'comp7_inv',
                 'comp8_inv',
                 'comp1_rate_percent_diff',
                 'comp2_rate_percent_diff',
                 'comp3_rate_percent_diff',
                 'comp4_rate_percent_diff',
                 'comp5_rate_percent_diff',
                 'comp6_rate_percent_diff',
                 'comp7_rate_percent_diff',
                 'comp8_rate_percent_diff'])
toc()

Elapsed time is 0.02692699432373047 seconds.


In [6]:
for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())


the number is missing values in visitor_hist_starrating is 47435
the number is missing values in visitor_hist_adr_usd is 47435
the number is missing values in prop_location_score2 is 10892
the number is missing values in srch_query_affinity_score is 46736
the number is missing values in orig_destination_distance is 15845
the number is missing values in gross_bookings_usd is 48623


# Daan's stuk

In [7]:
def orig_dest_distance_pandas_backup(visitor_country_id, prop_country_id, orig_destination_distance, i):
    orig = visitor_country_id[i]
    dest = prop_country_id[i]

    # check if someone has the exact same but with a distance
    others_pd = df.loc[df['visitor_location_country_id'] == orig]
    others = others_pd.loc[others_pd['prop_country_id'] == dest]

    if len(others) > 0 and others['orig_destination_distance'].isna().sum() < len(others):
        return np.mean(others['orig_destination_distance'])

    # only from same country
    others = df.loc[df['visitor_location_country_id'] == orig]

    if len(others) > 0 and others['orig_destination_distance'].isna().sum() < len(others):
        return np.mean(others['orig_destination_distance'])

    # only to same country
    others = df.loc[df['prop_country_id'] == dest]

    if len(others) > 0 and others['orig_destination_distance'].isna().sum() < len(others):
        return np.mean(others['orig_destination_distance'])
    
    return 1301

In [8]:
def orig_dest_distance(visitor_country_id, prop_country_id, orig_destination_distance, i):
    orig = visitor_country_id[i]
    dest = prop_country_id[i]

    # check if someone has the exact same but with a distance
    home = np.where(visitor_country_id == orig)
    destination = np.where(prop_country_id == dest)

    others_index = np.intersect1d(home, destination)
    others = np.take(orig_destination_distance, others_index)
    
    nans_np = np.count_nonzero(np.isnan(others))

    if len(others) > 0 and nans_np < len(others):
        return np.nanmean(others)

    # only from same country
    others = np.where(visitor_country_id == orig)
    others = np.take(orig_destination_distance, others)

    nans_np = np.count_nonzero(np.isnan(others))

    if len(others) > 0 and nans_np < len(others):
        return np.nanmean(others)

    # only to same country
    others = np.where(prop_country_id == dest)
    others = np.take(orig_destination_distance, others)

    nans_np = np.count_nonzero(np.isnan(others))
    if len(others) > 0 and nans_np < len(others):
        return np.nanmean(others)
    
    return 1301

In [9]:
def missing_distance(visitor_country_id, prop_country_id, orig_destination_distance):
    for i in tqdm(range(len(orig_destination_distance))):

        if np.isnan(orig_destination_distance[i]):
            result = orig_dest_distance(visitor_country_id, prop_country_id, orig_destination_distance, i)
            orig_destination_distance[i] = result

    return orig_destination_distance

In [10]:
tic()
visitor_country_id = df['visitor_location_country_id'].to_numpy()
prop_country_id = df['prop_country_id'].to_numpy()
orig_destination_distance = df['orig_destination_distance'].to_numpy()

df['orig_destination_distance'] = missing_distance(visitor_country_id, prop_country_id, orig_destination_distance)

if df['orig_destination_distance'].isna().sum() != 0:
    print("distance still has nans!!!!")
toc()

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:21<00:00, 2374.26it/s]

Elapsed time is 21.076100826263428 seconds.


In [11]:
for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())


the number is missing values in visitor_hist_starrating is 47435
the number is missing values in visitor_hist_adr_usd is 47435
the number is missing values in prop_location_score2 is 10892
the number is missing values in srch_query_affinity_score is 46736
the number is missing values in gross_bookings_usd is 48623


# Karim's stuk

In [12]:
tic()
# Drop column 'prop_location_score2', and 'gross_bookings_usd' too many missing values.
df = df.drop(columns=['prop_location_score2', 'gross_bookings_usd'])
toc()

Elapsed time is 0.010999917984008789 seconds.


In [13]:
for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())

# Kolommen die we voor nu nog niet hebben aangepast        
# visitor hist starrating
# visitor hist adr usd


the number is missing values in visitor_hist_starrating is 47435
the number is missing values in visitor_hist_adr_usd is 47435
the number is missing values in srch_query_affinity_score is 46736


In [14]:
import copy
import dateparser
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

from numpy.core.numeric import NaN
from tqdm import tqdm


def trim_cols(data):
    '''
    removes columns with >95% missing data values
    '''
    print('Trimming columns with > 95% missing values:')

    df = data.copy()
    total_values = len(df.index)
    removed_cols=[]
    for col in tqdm(df, total = df.shape[1]):
        missing_values = sum(df[col].isnull()==True)
        if missing_values > .95 * total_values:
            removed_cols.append(col)
            df.drop(col, 1)

    print('Removed cols:')
    for col in removed_cols:
        print(col)
    print('\n')

    return df

def trim_dates(dates):
    '''
    replaces datetime column with separate month and daypart column
    '''
    print('Trimming datetime:')

    months, dayparts = [], []
    
    for i in tqdm(range(len(dates))):
        # split date and time
        datetime = dateparser.parse(dates[i])

        # retrieve month 
        month = datetime.date().month
        months.append(month)
                
        # retrieve daypart based on hour of the day (night = 1, morning = 2, afternoon = 3, noon = 4)
        hour = datetime.time().hour
        daypart = int(np.round(hour/6 + .5))

        dayparts.append(daypart)

    return months, dayparts


def round_halves(raw_values):
    '''
    rounds to halves
    '''
    values = np.nan_to_num(raw_values)
    
    # round ratings to halves, or add NA if data is missing
    for i in tqdm(range(len(values))):
        # progress bar    
        if i % (1/len(values)) == 0:
            print(i/len(values), '%', end='\r')
        
        values[i] = round(values[i] * 2) / 2
    
    return values


def trim_avg_spent(spent):
    '''
    trims average price spent per night for customers

    TODO: kijken of de distributie heel hoog is rondom een getal, en daar misschien meer 'bins' maken
    '''
    print('Trimming visitor_hist_adr_usd:')
    # spent = df['visitor_hist_adr_usd'].to_numpy()
    avg_spent = np.nan_to_num(spent)

    # round to every 50 dollars spent or add 'NA'
    for i in tqdm(range(len(avg_spent))):
        multiplier = round(avg_spent[i]/50)
        avg_spent[i] = multiplier * 50

    return avg_spent


def trim_loc_score(loc_score1):
    '''
    rounds location score to halves or adds 'NA' for missing values
    '''
    print('Trimming prop_location_scores:')
    score1 = np.nan_to_num(loc_score1)
#     score2 = np.nan_to_num(loc_score2)

    for i in tqdm(range(len(score1))):
        score1[i] = np.round(score1[i] * 2) / 2
#         score2[i] = np.round(score2[i] * 2) / 2    

    return score1


def trim_hist_price(price):
    '''
    rounds prices or add NA for values of 0
    '''
    print('Trimming prop_log_historical_price:')
    hist_price = np.nan_to_num(price)
    for i in tqdm(range(len(hist_price))):
        hist_price[i] = np.round(hist_price[i] * 2) / 2
        
    return hist_price


def trim_price(price):
    print('Trimming price_usd:')
    trimmed_price = np.nan_to_num(price)
    for i in tqdm(range(len(trimmed_price))):    
        trimmed_price[i] = np.round(trimmed_price[i])

    return trimmed_price
    

def trim_booking_window(bw):
    '''
    changes booking window from days to weeks
    '''
    print('Trimming srch_booking_window')
    booking_window = np.nan_to_num(bw)
    
    for i in tqdm(range(len(booking_window))):
        booking_window[i] = np.round(booking_window[i]/7)
        
    return booking_window


def trim_dest_dist(dd):
    '''
    divides distance to destination by 100 miles
    '''
    dest_dist = np.nan_to_num(dd)
    print('Trimming orig_destination_distance:')

    for i in tqdm(range(len(dest_dist))):
        dest_dist[i] = np.round(dest_dist[i]/100)
        
    return dest_dist

In [15]:

'''
trims the data, add
_________________________________________________________________
try-excepts have been added in each function to check whether the 
column has not been removed due to >95% of data values missing.
'''

# from trim_functions import *
# import time


def trim_data(df):
        
    # track time
    start_time = time.time()
    
    # trim dates
    dates = df['date_time'].to_numpy()    
    months, dayparts = trim_dates(dates)
    df['month'] = months
    df['daypart'] = dayparts
    
    # trim ratings
    print('Trimming visitor_hist_starrating:')
    ratings = df['visitor_hist_starrating'].to_numpy()
    ratings = round_halves(ratings)
    df['visitor_hist_starrating'] = ratings

    # trim avg spent
    spent = df['visitor_hist_adr_usd'].to_numpy()
    avg_spent = trim_avg_spent(spent)
    df['visitor_hist_adr_usd'] = avg_spent
    

    # trim location scores
    loc_score1 = df['prop_location_score1'].to_numpy()
    loc_score1 = trim_loc_score(loc_score1)
    df['prop_location_score1'] = loc_score1

    # trim hist price
    hist_price = df['prop_log_historical_price'].to_numpy()
    trimmed_hist_price = trim_hist_price(hist_price)
    df['prop_log_historical_price'] = trimmed_hist_price

    # trim price
    price = df['price_usd'].to_numpy()
    trimmed_price = trim_price(price)
    df['price_usd'] = trimmed_price

    # trim booking window
    booking_window = df['srch_booking_window'].to_numpy()
    trimmed_booking_window = trim_booking_window(booking_window)
    df['srch_booking_window'] = trimmed_booking_window

    # trim destination distance
    dest_dist = df['orig_destination_distance'].to_numpy()
    trimmed_dest_dist = trim_dest_dist(dest_dist)
    df['orig_destination_distance'] = trimmed_dest_dist
    
    
    print("--- trimming took %s seconds ---" % (round(time.time() - start_time)))
    
    return df
tic()
df = trim_data(df)
toc()


  0%|                                                                              | 1/50000 [00:00<1:43:53,  8.02it/s]

Trimming datetime:


  0%|                                                                                        | 0/50000 [00:00<?, ?it/s]

Trimming visitor_hist_starrating:
Trimming visitor_hist_adr_usd:


 29%|████████████████████▋                                                   | 14361/50000 [00:00<00:00, 142569.33it/s]

...

Trimming srch_booking_window


 31%|██████████████████████▎                                                 | 15492/50000 [00:00<00:00, 154494.71it/s]

Trimming orig_destination_distance:


100%|████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 141213.23it/s]

--- trimming took 119 seconds ---
Elapsed time is 119.42201495170593 seconds.


In [16]:
df = df.drop(columns=['date_time'])

for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())


the number is missing values in srch_query_affinity_score is 46736


In [17]:
# Save the processed datafile
df.to_csv('small_data/cleaned_data_50000.csv')